# 基于CNN的MNIST手写数字识别
## 1. 加载Overlay

In [1]:
import time
from pynq import Overlay
import numpy as np
from pynq import Xlnk
import struct
from scipy.misc import imread
import cv2

overlay = Overlay('./mnist_systolic.bit')
print("Overlay downloaded successfully!")

/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


Overlay downloaded successfully!


## 2. 定义IP核驱动及其他功能函数

In [2]:
systolic_array_ip = overlay.systolic_array_0
pool_ip = overlay.Pool_0
xlnk = Xlnk()

# 脉动阵列驱动函数
def RunSystolic(array, din_a, din_b, bias, out):
    array.write(0x10, din_a.shape[0])
    array.write(0x18, din_a.shape[1])
    array.write(0x20, din_b.shape[1])
    array.write(0x28, din_a.physical_address)
    array.write(0x30, din_b.physical_address)
    array.write(0x38, bias.physical_address)
    array.write(0x40, out.physical_address)
    array.write(0, (array.read(0) & 0x80) | 0x01)
    tp = array.read(0)
    while not ((tp >> 1) & 0x1):
        tp = array.read(0)

# 硬件加速的卷积函数
def hwConv(array, Kw, S, relu_en, pad, feat_in, W, bias, feat_out):
    if pad > 0:
        feat_in = np.pad(feat_in, ((0, 0), (pad, pad), (pad, pad)), "constant")
    
    core_size = Kw*Kw

    # 在PS端的DRAM中为IP核的输入输出数据分配存储空间
    buf_a = xlnk.cma_array(shape = (feat_out.shape[0], feat_in.shape[0]*core_size), cacheable = 0, dtype = np.float32)
    buf_b = xlnk.cma_array(shape = (feat_in.shape[0]*core_size, feat_out.shape[1]*feat_out.shape[2]), cacheable = 0, dtype = np.float32)
    buf_c = xlnk.cma_array(shape = (feat_out.shape[0], feat_out.shape[1]*feat_out.shape[2]), cacheable = 0, dtype = np.float32)
    
    # TODO: 调整卷积核与特征图，以适应脉动阵列
    buf_a = ???
    buf_b = ???

    # 利用硬件矩阵乘法实现卷积加速
    RunSystolic(array, buf_a, buf_b, bias, buf_c)
    
    # 调整输出特征图的形状
    feat_out = buf_c.reshape(feat_out.shape[0], feat_out.shape[1], feat_out.shape[2])

# 池化IP核驱动函数
def hwPool(pool, Kw, mode, feat_in, feat_out):
    pool.write(0x10, feat_in.shape[2])
    pool.write(0x18, feat_in.shape[0])
    pool.write(0x20, feat_in.shape[1])
    pool.write(0x28, Kw)
    pool.write(0x30, mode)
    pool.write(0x38, feat_in.physical_address)
    pool.write(0x40, feat_out.physical_address)
    pool.write(0, (pool.read(0) & 0x80) | 0x01)
    while not ((pool.read(0) >> 1) & 0x1):
        pass
                   
# 软件卷积
def swConv(Kw, S, relu_en, pad, feat_in, W, bias, feat_out):
    for ch_o in range(feat_out.shape[0]):   # Output Channel
        for r_o in range(feat_out.shape[1]):    # Output Height
            for c_o in range(feat_out.shape[2]):    # Output Width
                mid_sum = 0.0
                for r in range(Kw):
                    for c in range(Kw):
                        r_indx = r_o*S + r - pad
                        c_indx = c_o*S + c - pad
                        if 0 <= r_indx and r_indx < feat_in.shape[1] and 0 <= c_indx and c_indx < feat_in.shape[2]:
                            for ch_i in range(feat_in.shape[0]):    # Input Channel
                                mid_sum += feat_in[ch_i][r_indx][c_indx] * W[ch_o][ch_i][r][c]
                feat_out[ch_o][r_o][c_o] = mid_sum + bias[ch_o]
                if relu_en and feat_out[ch_o][r_o][c_o] < 0:
                    feat_out[ch_o][r_o][c_o] = 0
                    
# 软件池化
def swPool(Kw, mode, feat_in, feat_out):
    for ch in range(feat_out.shape[0]):   # Output Channel
        for r in range(feat_out.shape[1]):   # Output Height
            for c in range(feat_out.shape[2]):   # Output Width
                tmp = 0
                for i in range(Kw):
                    for j in range(Kw):
                        tmp1 = feat_in[ch][r*Kw + i][c*Kw + j]
                        if mode == 0:         # MEAN
                            tmp += tmp1
                        elif mode == 1:       # MIN
                            tmp = tmp1 if tmp1 < tmp else tmp
                        elif mode == 2:       # MAX
                            tmp = tmp1 if tmp1 > tmp else tmp
                feat_out[ch][r][c] = tmp/(Kw*Kw) if mode == 0 else tmp

def readbinfile(filename,size):
    f = open(filename, "rb")
    z=[]
    for j in range(size):
        data = f.read(4)
        data_float = struct.unpack("f", data)[0]
        z.append(data_float)
    f.close()
    z = np.array(z)
    return z

## 3. CNN各层输入输出的基本参数

In [3]:
##################################################
# Conv1
IN_CH1     = 1
IN_WIDTH1  = 28
IN_HEIGHT1 = 28

KERNEL_W1  = 3
STRIDE1    = 1
RELU_EN1   = 1

PADDING1   = int((KERNEL_W1 - 1)/2)

OUT_CH1     = 16
OUT_WIDTH1  = int((IN_WIDTH1  + 2*PADDING1 - KERNEL_W1)/STRIDE1 + 1) # 28
OUT_HEIGHT1 = int((IN_HEIGHT1 + 2*PADDING1 - KERNEL_W1)/STRIDE1 + 1) # 28

##################################################
# Pool1
MODE11      = 2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_CH11     = OUT_CH1      # 16
IN_WIDTH11  = OUT_WIDTH1   # 28
IN_HEIGHT11 = OUT_HEIGHT1  # 28

KERNEL_W11  = 2

OUT_CH11     = IN_CH11                       # 16
OUT_WIDTH11  = int(IN_WIDTH11 /KERNEL_W11)   # 14
OUT_HEIGHT11 = int(IN_HEIGHT11/KERNEL_W11)   # 14

##################################################
# Conv2
IN_CH2     = OUT_CH11          # 16
IN_WIDTH2  = OUT_WIDTH11       # 14
IN_HEIGHT2 = OUT_HEIGHT11      # 14

KERNEL_W2  = 3
STRIDE2    = 1
RELU_EN2   = 1

PADDING2 = int((KERNEL_W2 - 1)/2)

OUT_CH2     = 32
OUT_WIDTH2  = int((IN_WIDTH2  + 2*PADDING2 - KERNEL_W2)/STRIDE2 + 1) # 14
OUT_HEIGHT2 = int((IN_HEIGHT2 + 2*PADDING2 - KERNEL_W2)/STRIDE2 + 1) # 14

##################################################
# Pool2
MODE21      = 2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_CH21     = OUT_CH2       # 32
IN_WIDTH21  = OUT_WIDTH2    # 14
IN_HEIGHT21 = OUT_HEIGHT2   # 14

KERNEL_W21  = 2

OUT_CH21     = IN_CH21                      # 32
OUT_WIDTH21  = int(IN_WIDTH21 /KERNEL_W21)  # 7
OUT_HEIGHT21 = int(IN_HEIGHT21/KERNEL_W21)  # 7

##################################################
# FC1
IN_CH3     = OUT_CH21      # 32
IN_WIDTH3  = OUT_WIDTH21   # 7
IN_HEIGHT3 = OUT_HEIGHT21  # 7

KERNEL_W3  = 7
STRIDE3    = 1
RELU_EN3   = 1

OUT_CH3     = 128
OUT_WIDTH3  = int((IN_WIDTH3  - KERNEL_W3)/STRIDE3 + 1) # 1
OUT_HEIGHT3 = int((IN_HEIGHT3 - KERNEL_W3)/STRIDE3 + 1) # 1

##################################################
# FC2
IN_CH4     = OUT_CH3     # 128
IN_WIDTH4  = OUT_WIDTH3  # 1
IN_HEIGHT4 = OUT_HEIGHT3 # 1

KERNEL_W4  = 1
STRIDE4    = 1
RELU_EN4   = 1

OUT_CH4     = 10
OUT_WIDTH4  = int((IN_WIDTH4  - KERNEL_W4)/STRIDE4 + 1) # 1
OUT_HEIGHT4 = int((IN_HEIGHT4 - KERNEL_W4)/STRIDE4 + 1) # 1

## 4. 读取网络参数

In [4]:
# Input image
image = xlnk.cma_array(shape = (IN_CH1, IN_HEIGHT1, IN_WIDTH1), cacheable = 0, dtype = np.float32)

# Conv1
W_conv1 = xlnk.cma_array(shape = (OUT_CH1, IN_CH1, KERNEL_W1, KERNEL_W1), cacheable = 0, dtype = np.float32)
b_conv1 = xlnk.cma_array(shape = (OUT_CH1), cacheable = 0, dtype = np.float32)
h_conv1 = xlnk.cma_array(shape = (OUT_CH1, OUT_HEIGHT1, OUT_WIDTH1), cacheable = 0, dtype = np.float32)
h_pool1 = xlnk.cma_array(shape = (OUT_CH11, OUT_HEIGHT11, OUT_WIDTH11), cacheable = 0, dtype = np.float32)

# Conv2
W_conv2 = xlnk.cma_array(shape = (OUT_CH2, IN_CH2, KERNEL_W2, KERNEL_W2), cacheable = 0, dtype = np.float32)
b_conv2 = xlnk.cma_array(shape = (OUT_CH2), cacheable = 0, dtype = np.float32)
h_conv2 = xlnk.cma_array(shape = (OUT_CH2, OUT_HEIGHT2, OUT_WIDTH2), cacheable = 0, dtype = np.float32)
h_pool2 = xlnk.cma_array(shape = (OUT_CH21, OUT_HEIGHT21, OUT_WIDTH21), cacheable = 0, dtype = np.float32)

# FC1
W_fc1 = xlnk.cma_array(shape = (OUT_CH3, IN_CH3, KERNEL_W3, KERNEL_W3), cacheable = 0, dtype = np.float32)
b_fc1 = xlnk.cma_array(shape = (OUT_CH3), cacheable = 0, dtype = np.float32)
h_fc1 = xlnk.cma_array(shape = (OUT_CH3, OUT_HEIGHT3, OUT_WIDTH3), cacheable = 0, dtype = np.float32)

# FC2
W_fc2 = xlnk.cma_array(shape = (OUT_CH4, IN_CH4, KERNEL_W4, KERNEL_W4), cacheable = 0, dtype = np.float32)
b_fc2 = xlnk.cma_array(shape = (OUT_CH4), cacheable = 0, dtype = np.float32)
h_fc2 = xlnk.cma_array(shape = (OUT_CH4, OUT_HEIGHT4, OUT_WIDTH4), cacheable = 0, dtype = np.float32)

# Read weights and bias from pre-tranined file
print("Conv1:\tloading weight... ", end = "")

w_conv1 = readbinfile("./data/W_conv1.bin", KERNEL_W1*KERNEL_W1*IN_CH1*OUT_CH1)
w_conv1 = w_conv1.reshape((KERNEL_W1, KERNEL_W1, IN_CH1, OUT_CH1))
for r in range(KERNEL_W1):
    for c in range(KERNEL_W1):
        for ch_i in range(IN_CH1):
            for ch_o in range(OUT_CH1):
                W_conv1[ch_o][ch_i][r][c] = w_conv1[r][c][ch_i][ch_o]

print("done")
print("\tloading bias... ", end = "")
                
B_conv1 = readbinfile("./data/b_conv1.bin",OUT_CH1)
for i in range(OUT_CH1):
    b_conv1[i] = B_conv1[i]

print("done")
print("Conv2:\tloading weight... ", end = "")

w_conv2 = readbinfile("./data/W_conv2.bin", KERNEL_W2*KERNEL_W2*IN_CH2*OUT_CH2)
w_conv2 = w_conv2.reshape((KERNEL_W2, KERNEL_W2, IN_CH2, OUT_CH2))
for r in range(KERNEL_W2):
    for c in range(KERNEL_W2):
        for ch_i in range(IN_CH2):
            for ch_o in range(OUT_CH2):
                W_conv2[ch_o][ch_i][r][c] = w_conv2[r][c][ch_i][ch_o]

print("done")
print("\tloading bias... ", end = "")

B_conv2 = readbinfile("./data/b_conv2.bin",OUT_CH2)
for i in range(OUT_CH2):
    b_conv2[i] = B_conv2[i]

print("done")
print("FC1:\tloading weight... ", end = "")

w_fc1 = readbinfile("./data/W_fc1.bin", KERNEL_W3*KERNEL_W3*IN_CH3*OUT_CH3)
w_fc1 = w_fc1.reshape((KERNEL_W3, KERNEL_W3, IN_CH3, OUT_CH3))
for r in range(KERNEL_W3):
    for c in range(KERNEL_W3):
        for ch_i in range(IN_CH3):
            for ch_o in range(OUT_CH3):
                W_fc1[ch_o][ch_i][r][c] = w_fc1[r][c][ch_i][ch_o]

print("done")
print("\tloading bias... ", end = "")

B_fc1 = readbinfile("./data/b_fc1.bin", OUT_CH3)
for i in range(OUT_CH3):
    b_fc1[i] = B_fc1[i]

print("done")
print("FC2:\tloading weight... ", end = "")

w_fc2 = readbinfile("./data/W_fc2.bin", KERNEL_W4*KERNEL_W4*IN_CH4*OUT_CH4)
w_fc2 = w_fc2.reshape((KERNEL_W4, KERNEL_W4, IN_CH4, OUT_CH4))
for r in range(KERNEL_W4):
    for c in range(KERNEL_W4):
        for ch_i in range(IN_CH4):
            for ch_o in range(OUT_CH4):
                W_fc2[ch_o][ch_i][r][c] = w_fc2[r][c][ch_i][ch_o]

print("done")
print("\tloading bias... ", end = "")

B_fc2 = readbinfile("./data/b_fc2.bin",OUT_CH4)
for i in range(OUT_CH4):
    b_fc2[i] = B_fc2[i]

print("done")
print("CNN loaded successfully!")

Conv1:	loading weight... done
	loading bias... done
Conv2:	loading weight... done
	loading bias... done
FC1:	loading weight... done
	loading bias... done
FC2:	loading weight... done
	loading bias... done
CNN loaded successfully!


## 5. CNN软件推导

In [5]:
image1 = cv2.imread("./data/1.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32)

for ch in range(IN_CH1):
    for r in range(IN_HEIGHT1):
        for c in range(IN_WIDTH1):
            image[ch][r][c] = (255 - image1[r][c])/255

print("Finish reading image.")

pt0 = time.clock()

# Conv1
swConv(KERNEL_W1, STRIDE1, RELU_EN1, PADDING1, image, W_conv1, b_conv1, h_conv1)
swPool(KERNEL_W11, MODE11, h_conv1, h_pool1)
# Conv2
swConv(KERNEL_W2, STRIDE2, RELU_EN2, PADDING2, h_pool1, W_conv2, b_conv2, h_conv2)
swPool(KERNEL_W21, MODE21, h_conv2, h_pool2)
# FC1
swConv(KERNEL_W3, STRIDE3, RELU_EN3, 0, h_pool2, W_fc1, b_fc1, h_fc1)
# FC2
swConv(KERNEL_W4, STRIDE4, RELU_EN4, 0, h_fc1, W_fc2, b_fc2, h_fc2)

pt1 = time.clock()
time_sw = pt1 - pt0

print("Software inference done.")
print("Time consumed: %fs" % time_sw)

MAX = h_fc2[0][0][0]
result = 0
for ch in range(1, OUT_CH4):
    if (h_fc2[ch][0][0] > MAX):
        MAX = h_fc2[ch][0][0]
        result = ch

print("The image was recognized as " + str(result))

Finish reading image.
Software inference done.
Time consumed: 751.041182s
The image was recognized as 2


## 6. 利用硬件加速推导

In [6]:
pt0 = time.clock()

# Conv1
hwConv(systolic_array_ip, KERNEL_W1, STRIDE1, RELU_EN1, PADDING1, image, W_conv1, b_conv1, h_conv1)
hwPool(pool_ip, KERNEL_W11, MODE11, h_conv1, h_pool1)
# Conv2
hwConv(systolic_array_ip, KERNEL_W2, STRIDE2, RELU_EN2, PADDING2, h_pool1, W_conv2, b_conv2, h_conv2)
hwPool(pool_ip, KERNEL_W21, MODE21, h_conv2, h_pool2)
# FC1
hwConv(systolic_array_ip, KERNEL_W3, STRIDE3, RELU_EN3, 0, h_pool2, W_fc1, b_fc1, h_fc1)
# FC2
hwConv(systolic_array_ip, KERNEL_W4, STRIDE4, RELU_EN4, 0, h_fc1, W_fc2, b_fc2, h_fc2)

pt1 = time.clock()
time_hw = pt1 - pt0

print("Hardware-accelerated inference done.")
print("Time consumed: %fs" % time_hw)
print("Speedup: %.2f" % (time_sw/time_hw))

MAX = h_fc2[0][0][0]
result = 0
for ch in range(1, OUT_CH4):
    if (h_fc2[ch][0][0] > MAX):
        MAX = h_fc2[ch][0][0]
        result = ch

print("The image was recognized as " + str(result))

Hardware-accelerated inference done.
Time consumed: 0.911397s
Speedup: 824.05
The image was recognized as 2
